In [1]:
import sys
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.cluster import KMeans

In [2]:
lambert_crs = ccrs.LambertConformal(
    central_longitude=-107.0,
    central_latitude=50.0,
    standard_parallels=[50.0, 50.0],
    false_easting=5632642.22547,
    false_northing=4612545.65137
)

In [ ]:
air = xr.open_dataset("../data/NARR/air.200009.nc")
uwnd = xr.open_dataset("../data/NARR/uwnd.200009.nc")
vwnd = xr.open_dataset("../data/NARR/vwnd.200009.nc")
shum = xr.open_dataset("../data/NARR/shum.200009.nc")
omega = xr.open_dataset("../data/NARR/omega.200009.nc")

full = xr.merge([air,shum,omega,uwnd,vwnd])

In [4]:
def subset_by_latlon(data, lon_min, lon_max, lat_min, lat_max):

    lon = data['lon']
    lat = data['lat']

    mask = (lon >= lon_min) & (lon <= lon_max) & (lat >= lat_min) & (lat <= lat_max)
    y_indices, x_indices = np.where(mask)

    x_min, x_max = x_indices.min(), x_indices.max()
    y_min, y_max = y_indices.min(), y_indices.max()

    subset = data.isel(x=slice(x_min, x_max + 1), y=slice(y_min, y_max + 1))

    return subset

def join_variables(dataset: xr.Dataset):
    grid_size = dataset.sizes['y'] * dataset.sizes['x']
    n_lvl = dataset.sizes['level']
    arr = dataset.to_array()
    output = np.array(arr).reshape(6,n_lvl,grid_size).reshape(6*n_lvl,grid_size).T
    return output

def slice_time(dataset: xr.Dataset, time: datetime):
    output = dataset.sel(time=time, method="nearest")
    return output

In [12]:
full = full.stack(location=('y', 'x')).sel(time=full.time[0],method="nearest")
full = full.drop_vars(['Lambert_Conformal']).to_stacked_array("var-lvl",sample_dims=["location"])

In [13]:
full

<xarray.DataArray 'air' (location: 96673, var-lvl: 145)> Size: 56MB
array([[ 2.9596826e+02,  2.9425000e+02,  2.9305200e+02, ...,
        -1.4444885e+00,  3.4071350e-01, -2.6464844e-01],
       [ 2.9640576e+02,  2.9460938e+02,  2.9325513e+02, ...,
        -1.4444885e+00,  3.4071350e-01, -2.6464844e-01],
       [ 2.9684326e+02,  2.9490625e+02,  2.9347388e+02, ...,
        -1.7257385e+00,  4.3838501e-02, -3.2714844e-01],
       ...,
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan]], dtype=float32)
Coordinates:
    time      datetime64[ns] 8B 2000-09-01
    lat       (location) float32 387kB 1.0 1.104 1.208 ... 46.93 46.64 46.35
    lon       (location) float32 387kB -145.5 -145.3 -145.1 ... -2.644 -2.57
  * location  (location) object 773kB MultiIndex
  * y         (location) float32 387kB 0.0 0.0 0.0 ... 8.96e+06 8.96e+06
  * x         (location) float32 387kB 0.0 3.246e+04 ... 1.126e+07 1.13e+07
  * var-lvl   (var-lvl) object 1kB MultiIndex
  * variable  (var-lvl) <U5 3kB 'air' 'air' 'air' 'air' ... 'vwnd' 'vwnd' 'vwnd'
  * level     (var-lvl) float32 580B 1e+03 975.0 950.0 ... 150.0 125.0 100.0
Attributes: (12/13)
    GRIB_id:        11
    GRIB_name:      TMP
    dataset:        NARR 3-hourly
    grid_mapping:   Lambert_Conformal
    level_desc:     Pressure Levels
    long_name:      3-hourly Air Temperature on Pressure Levels
    ...             ...
    standard_name:  air_temperature
    statistic:      Individual Obs
    units:          degK
    valid_range:    [137.5 362.5]
    var_desc:       Air temperature
    actual_range:   [189.88913 316.97607]